In [4]:
import os, random, glob, pickle, collections, math, json, time
import numpy as np
import pandas as pd
from __future__ import division
from __future__ import print_function
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
%matplotlib inline 

from keras.models import Sequential, Model, load_model
from keras.layers import GlobalAveragePooling2D, Flatten, Dropout, Dense, LeakyReLU
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.preprocessing import image
from keras import backend as K
K.set_image_dim_ordering('tf')

In [17]:
TEST_DIR = '../RFCN/JPEGImages/'
RFCN_MODEL = 'resnet101_rfcn_ohem_iter_30000'
CROP_MODEL = 'resnet50_FT38_Hybrid_wNoF2'
if os.getcwd().split('/')[-1] != CROP_MODEL:
    print('WRONG CROP_MODEL DIR!!!')
CHECKPOINT_DIR = './checkpoint/'
if not os.path.exists(CHECKPOINT_DIR):
    os.mkdir(CHECKPOINT_DIR)
LOG_DIR = './log/'
if not os.path.exists(LOG_DIR):
    os.mkdir(LOG_DIR)
OUTPUT_DIR = './output/'
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)
FISH_CLASSES = ['NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']
CONF_THRESH = 0.8
ROWS = 224
COLS = 224
BATCHSIZE = 128
LEARNINGRATE = 1e-4

def featurewise_center(x):
    mean = np.mean(x, axis=0, keepdims=True)
    mean = np.mean(mean, axis=(1,2), keepdims=True)
    x_centered = x - mean
    return x_centered    

def featurewise_mean(x):
    mean = np.mean(x, axis=0)
    mean = np.mean(mean, axis=(0,1))
    return mean

def load_img(path, bbox, target_size=None):
    img = Image.open(path)
#     img = img.convert('RGB')
    cropped = img.crop((bbox[0],bbox[1],bbox[2],bbox[3]))
    width_cropped, height_cropped = cropped.size
    if height_cropped > width_cropped: cropped = cropped.transpose(method=2)
    image_name = 'temp_{:f}.jpg'.format(time.time())  
    if target_size:
        cropped = cropped.resize((target_size[1], target_size[0]), Image.BILINEAR)
    return cropped

def get_best_model(checkpoint_dir = CHECKPOINT_DIR):
    files = glob.glob(checkpoint_dir+'*')
    val_losses = [float(f.split('-')[-1][:-5]) for f in files]
    index = val_losses.index(min(val_losses))
    print('Loading model from checkpoint file ' + files[index])
    model = load_model(files[index])
    model_name = files[index].split('/')[-1]
    print('Loading model Done!')
    return (model, model_name)

def data_from_df(df):
    X = np.ndarray((df.shape[0], ROWS, COLS, 3), dtype=np.uint8)
    y = np.zeros((df.shape[0], len(FISH_CLASSES)), dtype=K.floatx())
    i = 0
    for index,row in df.iterrows():
        image_file = row['image_file']
        fish = row['crop_class']
        bbox = [row['xmin'],row['ymin'],row['xmax'],row['ymax']]
        cropped = load_img(TEST_DIR+image_file,bbox,target_size=(ROWS,COLS))
        X[i] = np.asarray(cropped)
        y[i,FISH_CLASSES.index(fish)] = 1
        i += 1
    return (X, y)

def data_load(name):
    file_name = 'data_'+name+'_{}_{}.pickle'.format(ROWS, COLS)
    if os.path.exists(OUTPUT_DIR+file_name):
        print ('Loading from file '+file_name)
        with open(OUTPUT_DIR+file_name, 'rb') as f:
            data = pickle.load(f)
        X = data['X']
        y = data['y']
    else:
        print ('Generating file '+file_name)
        
        if name=='train' or name=='valid': 
            df = GTbbox_df[GTbbox_df['split']==name]
        elif name=='all':
            df = GTbbox_df
        else:
            print('Invalid name '+name)
    
        X, y = data_from_df(df)

        data = {'X': X,'y': y}
        with open(OUTPUT_DIR+file_name, 'wb') as f:
            pickle.dump(data, f)
    X = X.astype(np.float32)
    X /= 255.
    return (X, y)

In [18]:
# GTbbox_df = ['image_file','image_class','crop_index','crop_class','xmin',''ymin','xmax','ymax','split']

file_name = 'GTbbox_df.pickle'
if os.path.exists(OUTPUT_DIR+file_name):
    print ('Loading from file '+file_name)
    GTbbox_df = pd.read_pickle(OUTPUT_DIR+file_name)
else:
    print ('Generating file '+file_name)       
    GTbbox_df = pd.DataFrame(columns=['image_file','image_class','crop_index','crop_class','xmin','ymin','xmax','ymax'])  

    crop_classes=FISH_CLASSES[:]
    crop_classes.remove('NoF')
    with open("../RFCN/ImageSets/Main/train_test.txt","r") as f:
        train_file_labels = f.readlines()
        
    for c in crop_classes:
        print(c)
        j = json.load(open('../data/BBannotations/{}.json'.format(c), 'r'))
        for l in j: 
            filename = l["filename"]
            head, image_file = os.path.split(filename)
            basename, file_extension = os.path.splitext(image_file)
            for i in range(len(train_file_labels)):
                if train_file_labels[i][:9] == basename:
                    image_class = train_file_labels[i][10:-1]
                    break
            image = Image.open(TEST_DIR+'/'+image_file)
            width_image, height_image = image.size
            for i in range(len(l["annotations"])):
                a = l["annotations"][i]
                xmin = (a["x"])
                ymin = (a["y"])
                width = (a["width"])
                height = (a["height"])
                xmax = xmin + width
                ymax = ymin + height
                assert max(xmin,0)<min(xmax,width_image)
                assert max(ymin,0)<min(ymax,height_image)
                GTbbox_df.loc[len(GTbbox_df)]=[image_file,image_class,i,a["class"],max(xmin,0),max(ymin,0),min(xmax,width_image),min(ymax,height_image)] 
        
    print('NoF')  
    num_NoF = GTbbox_df.shape[0]*3
    RFCN_MODEL = 'resnet101_rfcn_ohem_iter_30000'
    with open('../data/RFCN_detections/detections_full_AGNOSTICnms_'+RFCN_MODEL+'.pkl','rb') as f:
        detections_full_AGNOSTICnms = pickle.load(f) 
    train_detections_full_AGNOSTICnms = detections_full_AGNOSTICnms[1000:]
    with open("../RFCN/ImageSets/Main/test.txt","r") as f:
        test_files = f.readlines()
    train_files = test_files[1000:]
    num_NoF_perIm = int(math.ceil(num_NoF / len(train_detections_full_AGNOSTICnms)))

    for im in range(len(train_files)):
        image_file = train_files[im][:9]+'.jpg'
        for i in range(len(train_file_labels)):
            if train_file_labels[i][:9] == train_files[im][:9]:
                image_class = train_file_labels[i][10:-1]
                break
        image = Image.open(TEST_DIR+image_file)
        width_image, height_image = image.size
        detects_im = train_detections_full_AGNOSTICnms[im]
        detects_im = detects_im[np.where(detects_im[:,4] >= 0.999)]
        np.random.seed(1986)
        bboxes = detects_im[np.random.choice(detects_im.shape[0], num_NoF_perIm, replace=False), :]
        for j in range(bboxes.shape[0]):    
            bbox = bboxes[j]
            xmin = bbox[0]
            ymin = bbox[1]
            xmax = bbox[2]
            ymax = bbox[3]
            assert max(xmin,0)<min(xmax,width_image)
            assert max(ymin,0)<min(ymax,height_image)
            GTbbox_df.loc[len(GTbbox_df)]=[image_file,image_class,j,'NoF',xmin,ymin,xmax,ymax]

    image_files_df = GTbbox_df[['image_file','image_class']]
    image_files_df.drop_duplicates(inplace = True)
    test_file_nb = image_files_df.shape[0] - int(math.ceil(image_files_df.shape[0]*0.9))
    train_files_df, valid_files_df = train_test_split(image_files_df, test_size=test_file_nb, random_state=1986, stratify=image_files_df.image_class)
    GTbbox_df['split'] = ['train' if image_file in train_files_df.image_file.tolist() else 'valid' for image_file in GTbbox_df.image_file]

    GTbbox_df.to_pickle(OUTPUT_DIR+file_name)
    print('Done')

# GTbbox_df.loc[(GTbbox_df['image_class']!='NoF') & (GTbbox_df['crop_class']!='NoF') & (GTbbox_df['image_class']!=GTbbox_df['crop_class'])]

Loading from file GTbbox_df.pickle


In [7]:
#Load data
X_all, y_all = data_load('all')
X_train, y_train = data_load('train')
X_valid, y_valid = data_load('valid')  
print('Loading data done.')

print('all sample ', X_all.shape[0])
print('train sample ', X_train.shape[0])
print('valid sample ', X_valid.shape[0])

print('featurewise mean of X_all is ', featurewise_mean(X_all))
print('featurewise mean of X_train is ', featurewise_mean(X_train))
print('featurewise mean of X_valid is ', featurewise_mean(X_valid))

Loading from file data_all_224_224.pickle
Loading from file data_train_224_224.pickle
Loading from file data_valid_224_224.pickle
Loading data done.
all sample  19479
train sample  17551
valid sample  1928
featurewise mean of X_all is  [ 0.40888795  0.44883755  0.41426802]
featurewise mean of X_train is  [ 0.40796331  0.44811365  0.4137122 ]
featurewise mean of X_valid is  [ 0.41731915  0.45542452  0.41931435]


In [8]:
# #class weight = n_samples / (n_classes * np.bincount(y))
# class_weight_fish = dict(GTbbox_df.groupby('crop_class').size())
# class_weight = {}
# n_samples = GTbbox_df.shape[0]
# for key,value in class_weight_fish.items():
#         class_weight[CROP_CLASSES.index(key)] = n_samples / (len(CROP_CLASSES)*value)
# class_weight

train_df = GTbbox_df.loc[GTbbox_df['split']=='train']
class_weight_fish = dict(train_df.groupby('crop_class').size())
class_weight = {}
ref = max(class_weight_fish.values())
for key,value in class_weight_fish.items():
    class_weight[FISH_CLASSES.index(key)] = ref/value
class_weight

{0: 1.0,
 1: 5.9885512989872307,
 2: 48.571428571428569,
 3: 119.29824561403508,
 4: 144.68085106382978,
 5: 45.182724252491695,
 6: 79.532163742690059,
 7: 18.888888888888889}

In [19]:
#data augmentation

all_mean = featurewise_mean(X_all)
def preprocessing_function(x):
#     resnet50 image preprocessing
#     'RGB'->'BGR'
#     x = x[:, :, ::-1]
    x -= np.reshape(all_mean, [1, 1, 3])
    return x

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocessing_function,
    rotation_range=180,
    shear_range=0.2,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True)
train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCHSIZE, shuffle=True, seed=None)
steps_per_epoch = int(math.floor(X_train.shape[0]/BATCHSIZE))

X_valid_centered = X_valid - np.reshape(all_mean, [1, 1, 1, 3])


In [11]:
#callbacks

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=25, verbose=1, mode='auto')        

model_checkpoint = ModelCheckpoint(filepath=CHECKPOINT_DIR+'weights.{epoch:03d}-{val_loss:.4f}.hdf5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
        
learningrate_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='auto', epsilon=0.001, cooldown=0, min_lr=0)

tensorboard = TensorBoard(log_dir=LOG_DIR, histogram_freq=0, write_graph=False, write_images=True)


In [14]:
#Resnet50
#top layer training

from keras.applications.resnet50 import ResNet50

base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
# x = Dropout(0.5)(x)
predictions = Dense(len(FISH_CLASSES), kernel_initializer='glorot_normal', activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
optimizer = Adam(lr=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# train the model on the new data for a few epochs
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=30, verbose=1, 
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=(X_valid_centered, y_valid), 
                    class_weight=class_weight, workers=3, pickle_safe=True)

Epoch 1/30
137/137 [==============================] - 230s - loss: 12.1730 - acc: 0.3928 - val_loss: 1.1088 - val_acc: 0.7822
Epoch 2/30
137/137 [==============================] - 227s - loss: 9.7741 - acc: 0.6222 - val_loss: 0.9203 - val_acc: 0.7822
Epoch 3/30
137/137 [==============================] - 227s - loss: 8.4704 - acc: 0.6771 - val_loss: 0.9270 - val_acc: 0.7822
Epoch 4/30
137/137 [==============================] - 223s - loss: 7.5488 - acc: 0.7087 - val_loss: 0.9765 - val_acc: 0.7822
Epoch 5/30
137/137 [==============================] - 227s - loss: 6.9289 - acc: 0.7316 - val_loss: 0.8801 - val_acc: 0.7822
Epoch 6/30
137/137 [==============================] - 227s - loss: 6.9678 - acc: 0.7308 - val_loss: 0.7690 - val_acc: 0.7858
Epoch 7/30
137/137 [==============================] - 223s - loss: 6.1640 - acc: 0.7489 - val_loss: 0.6059 - val_acc: 0.8076
Epoch 8/30
137/137 [==============================] - 228s - loss: 6.0128 - acc: 0.7595 - val_loss: 0.5644 - val_acc: 0.8138

In [15]:
### Resnet50
# fine tuning
# 164 conv5c+top
# 142 conv5+top
# 80 conv4+conv5+top
# 38 conv3+conv4+conv5+top
start_layer = 38

model, model_name = get_best_model()
# print('Loading model from weights.004-0.0565.hdf5')
# model = load_model(CHECHPOINT_DIR+'weights.004-0.0565.hdf5')

for layer in model.layers[:start_layer]:
   layer.trainable = False
for layer in model.layers[start_layer:]:
   layer.trainable = True
d
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
optimizer = Adam(lr=1e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=300, verbose=1, 
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=(X_valid_centered, y_valid), 
                    class_weight=class_weight, workers=3, pickle_safe=True, initial_epoch=30)

Loading model from checkpoint file ./checkpoint/weights.029-0.3772.hdf5
Loading model Done!
Epoch 31/300
137/137 [==============================] - 447s - loss: 3.4249 - acc: 0.8466 - val_loss: 0.3305 - val_acc: 0.8911
Epoch 32/300
137/137 [==============================] - 432s - loss: 2.7443 - acc: 0.8637 - val_loss: 0.2951 - val_acc: 0.9082
Epoch 33/300
137/137 [==============================] - 433s - loss: 2.4527 - acc: 0.8821 - val_loss: 0.2551 - val_acc: 0.9238
Epoch 34/300
137/137 [==============================] - 425s - loss: 1.9716 - acc: 0.9005 - val_loss: 0.2499 - val_acc: 0.9206
Epoch 35/300
137/137 [==============================] - 433s - loss: 1.8214 - acc: 0.9076 - val_loss: 0.2359 - val_acc: 0.9321
Epoch 36/300
137/137 [==============================] - 433s - loss: 1.7055 - acc: 0.9180 - val_loss: 0.2277 - val_acc: 0.9378
Epoch 37/300
137/137 [==============================] - 426s - loss: 1.4213 - acc: 0.9244 - val_loss: 0.2068 - val_acc: 0.9388
Epoch 38/300
137/13

In [ ]:
#Resnet50
#top layer training to end

from keras.applications.resnet50 import ResNet50

base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
# x = Dropout(0.5)(x)
predictions = Dense(len(FISH_CLASSES), kernel_initializer='glorot_normal', activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
optimizer = Adam(lr=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print('BATCHSIZE', BATCHSIZE)
# train the model on the new data for a few epochs
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=300, verbose=1, 
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=(X_valid_centered, y_valid), 
                    class_weight=class_weight, workers=3, pickle_safe=True)

BATCHSIZE 128
Epoch 1/300
137/137 [==============================] - 246s - loss: 12.7712 - acc: 0.3464 - val_loss: 1.4204 - val_acc: 0.7822
Epoch 2/300
 78/137 [================>.............] - ETA: 80s - loss: 10.5625 - acc: 0.5713 

In [ ]:
### Resnet50
# fine tuning
# 164 conv5c+top
# 142 conv5+top
# 80 conv4+conv5+top
# 38 conv3+conv4+conv5+top
# 6 conv2+conv3+conv4+conv5+top
# 2 conv1+conv2+conv3+conv4+conv5+top i.e. FROM SCRATCH
start_layer = 2

# model, model_name = get_best_model()
model = load_model(CHECKPOINT_DIR + 'weights.029-0.3772.hdf5')
model_name = 'weights.029-0.3772.hdf5'
print('model_name', model_name)

for layer in model.layers[:start_layer]:
   layer.trainable = False
for layer in model.layers[start_layer:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
optimizer = Adam(lr=1e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print('BATCHSIZE', BATCHSIZE)
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=300, verbose=1, 
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=(X_valid_centered, y_valid), 
                    class_weight=class_weight, workers=3, pickle_safe=True, initial_epoch=30)

In [8]:
#resume training

# model, model_name = get_best_model()
model = load_model(CHECKPOINT_DIR + 'weights.084-0.0838.hdf5')
model_name = 'weights.084-0.0838.hdf5'
print('model_name', model_name)

# #try increasing learningrate
# optimizer = Adam(lr=1e-4)
# model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=300, verbose=1, 
                    callbacks=[early_stopping, model_checkpoint, learningrate_schedule, tensorboard], 
                    validation_data=(X_valid_centered, y_valid), 
                    class_weight=class_weight, workers=3, pickle_safe=True, initial_epoch=85)

model_name weights.084-0.0838.hdf5
Epoch 86/300
137/137 [==============================] - 420s - loss: 0.0769 - acc: 0.9917 - val_loss: 0.0838 - val_acc: 0.9746
Epoch 87/300
137/137 [==============================] - 405s - loss: 0.0714 - acc: 0.9921 - val_loss: 0.0842 - val_acc: 0.9735
Epoch 88/300
137/137 [==============================] - 402s - loss: 0.0829 - acc: 0.9893 - val_loss: 0.0850 - val_acc: 0.9751
Epoch 89/300
137/137 [==============================] - 400s - loss: 0.0663 - acc: 0.9917 - val_loss: 0.0842 - val_acc: 0.9761
Epoch 90/300
137/137 [==============================] - 404s - loss: 0.0740 - acc: 0.9917 - val_loss: 0.0837 - val_acc: 0.9761
Epoch 91/300
137/137 [==============================] - 408s - loss: 0.0712 - acc: 0.9909 - val_loss: 0.0821 - val_acc: 0.9761
Epoch 92/300
137/137 [==============================] - 400s - loss: 0.1098 - acc: 0.9919 - val_loss: 0.0830 - val_acc: 0.9761
Epoch 93/300
137/137 [==============================] - 404s - loss: 0.0689 

In [5]:
#test prepare

test_model, test_model_name = get_best_model()
# test_model = load_model('./resnet50_FT38_CW_STGTrain/checkpoint/weights.000-0.0327.hdf5')
# test_model_name = 'weights.000-0.0327.hdf5'
# print('test_model_name', test_model_name)

def preprocess_featurewise(x, featurewise_mean):
#     resnet50 image preprocessing
#     'RGB'->'BGR'
#     x = x[:, :, ::-1]
    x -= np.reshape(featurewise_mean, [1, 1, 3])
    return x

def test_generator(df, mean, datagen = None, batch_size = BATCHSIZE):
    n = df.shape[0]
    batch_index = 0
    while 1:
        current_index = batch_index * batch_size
        if n >= current_index + batch_size:
            current_batch_size = batch_size
            batch_index += 1    
        else:
            current_batch_size = n - current_index
            batch_index = 0        
        batch_df = df[current_index:current_index+current_batch_size]
        batch_x = np.zeros((batch_df.shape[0], ROWS, COLS, 3), dtype=K.floatx())
        i = 0
        for index,row in batch_df.iterrows():
            image_file = row['image_file']
            bbox = [row['xmin'],row['ymin'],row['xmax'],row['ymax']]
            cropped = load_img(TEST_DIR+image_file,bbox,target_size=(ROWS,COLS))
            x = np.asarray(cropped, dtype=K.floatx())
            x /= 255.
            if datagen is not None: x = datagen.random_transform(x)            
            x = preprocess_featurewise(x, mean)
            batch_x[i] = x
            i += 1
        if batch_index%50 == 0: print('batch_index', batch_index)
        yield(batch_x)
        
test_aug_datagen = ImageDataGenerator(
    rotation_range=180,
    shear_range=0.2,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True)


Loading model from checkpoint file ./checkpoint/weights.139-0.0777.hdf5
Loading model Done!


In [6]:
# train_mean = [0.37698776,  0.41491762,  0.38681713]
all_mean = featurewise_mean(X_all)
print('all_mean:', all_mean)

all_mean: [ 0.40888795  0.44883755  0.41426802]


In [74]:
#GTbbox_CROPpred_df = ['image_file','crop_index','crop_class','xmin','ymin','xmax','ymax','split'
#                      'NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT', 'logloss',
#                      'ALB_woNoF', 'BET_woNoF', 'DOL_woNoF', 'LAG_woNoF', 'OTHER_woNoF', 'SHARK_woNoF', 'YFT_woNoF', 'logloss_woNoF']

file_name = 'GTbbox_CROPpred_df_'+test_model_name+'_.pickle'
if os.path.exists(OUTPUT_DIR+file_name):
    print ('Loading from file '+file_name)
    GTbbox_CROPpred_df = pd.read_pickle(OUTPUT_DIR+file_name)
else:
    print ('Generating file '+file_name) 
    nb_augmentation = 1
    if nb_augmentation ==1:
        test_preds = test_model.predict_generator(test_generator(df=GTbbox_df, mean=all_mean), 
                                                  steps=int(math.ceil(GTbbox_df.shape[0]/BATCHSIZE)), workers=1, pickle_safe=False)
    else:
        test_preds = np.zeros((GTbbox_df.shape[0], len(FISH_CLASSES)), dtype=K.floatx())
        for idx in range(nb_augmentation):
            print('{}th augmentation for testing ...'.format(idx+1))
            test_preds += test_model.predict_generator(test_generator(df=GTbbox_df, mean=all_mean, datagen=test_aug_datagen), 
                                                       steps=int(math.ceil(GTbbox_df.shape[0]/BATCHSIZE)), workers=1, pickle_safe=False)
        test_preds /= nb_augmentation

    CROPpred_df = pd.DataFrame(test_preds, columns=FISH_CLASSES)
    GTbbox_CROPpred_df = pd.concat([GTbbox_df,CROPpred_df], axis=1)
    GTbbox_CROPpred_df['logloss'] = GTbbox_CROPpred_df.apply(lambda row: -math.log(row[row['crop_class']]), axis=1)
    
    for fish in FISH_CLASSES:
        GTbbox_CROPpred_df[fish+'_woNoF'] = GTbbox_CROPpred_df.apply(lambda row: row[fish]/(1-row['NoF']) if fish!='NoF' else np.inf , axis=1)
    GTbbox_CROPpred_df['logloss_woNoF'] = GTbbox_CROPpred_df.apply(lambda row: -math.log(row[row['crop_class']+'_woNoF']), axis=1)
    
    GTbbox_CROPpred_df.to_pickle(OUTPUT_DIR+file_name) 
    
crop_valid_loss = GTbbox_CROPpred_df.loc[(GTbbox_CROPpred_df['crop_class']!='NoF') & (GTbbox_CROPpred_df['split']=='valid'), 'logloss_woNoF'].mean()
print('crop_valid_loss:', crop_valid_loss)
crop_all_loss = GTbbox_CROPpred_df.loc[GTbbox_CROPpred_df['crop_class']!='NoF', 'logloss_woNoF'].mean()
print('crop_all_loss:', crop_all_loss)

Loading from file GTbbox_CROPpred_df_weights.139-0.0777.hdf5_.pickle
crop_valid_loss: 0.103820501238
crop_all_loss: 0.0195811475679


In [78]:
print('all loss:', GTbbox_CROPpred_df['logloss'].mean())
print(GTbbox_CROPpred_df.groupby(['crop_class'])['logloss'].mean())
print('all crop loss:', GTbbox_CROPpred_df[GTbbox_CROPpred_df['crop_class']!='NoF']['logloss_woNoF'].mean())
print(GTbbox_CROPpred_df[GTbbox_CROPpred_df['crop_class']!='NoF'].groupby(['crop_class'])['logloss_woNoF'].mean())

train_CROPpred_df = GTbbox_CROPpred_df[GTbbox_CROPpred_df['split']=='train']
print('train loss:', train_CROPpred_df['logloss'].mean())
print('train crop loss:', train_CROPpred_df[train_CROPpred_df['crop_class']!='NoF']['logloss_woNoF'].mean())
print(train_CROPpred_df[train_CROPpred_df['crop_class']!='NoF'].groupby(['crop_class'])['logloss_woNoF'].mean())

valid_CROPpred_df = GTbbox_CROPpred_df[GTbbox_CROPpred_df['split']=='valid']
print('valid loss:', valid_CROPpred_df['logloss'].mean())
print('valid crop loss:', valid_CROPpred_df[valid_CROPpred_df['crop_class']!='NoF']['logloss_woNoF'].mean())
print(valid_CROPpred_df[valid_CROPpred_df['crop_class']!='NoF'].groupby(['crop_class'])['logloss_woNoF'].mean())

all loss: 0.0287306753994
crop_class
ALB      0.043330
BET      0.084191
DOL      0.002389
LAG      0.000585
NoF      0.026685
OTHER    0.013770
SHARK    0.001024
YFT      0.020902
Name: logloss, dtype: float64
all crop loss: 0.0195811475679
crop_class
ALB      0.016577
BET      0.083910
DOL      0.001413
LAG      0.000519
OTHER    0.010584
SHARK    0.000208
YFT      0.018096
Name: logloss_woNoF, dtype: float64
train loss: 0.0233468814897
train crop loss: 0.0106263187798
crop_class
ALB      0.012814
BET      0.014375
DOL      0.000259
LAG      0.000567
OTHER    0.002018
SHARK    0.000227
YFT      0.011293
Name: logloss_woNoF, dtype: float64
valid loss: 0.0777405130083
valid crop loss: 0.103820501238
crop_class
ALB      0.051892
BET      0.832748
DOL      0.012374
LAG      0.000106
OTHER    0.091162
SHARK    0.000021
YFT      0.080101
Name: logloss_woNoF, dtype: float64


In [9]:
# RFCNbbox_RFCNpred_df = ['image_class','image_file','crop_index','xmin','ymin','xmax','ymax',
#                          'NoF_RFCN', 'ALB_RFCN', 'BET_RFCN', 'DOL_RFCN',
#                          'LAG_RFCN', 'OTHER_RFCN', 'SHARK_RFCN', 'YFT_RFCN']
# select fish_conf >= CONF_THRESH

file_name = 'RFCNbbox_RFCNpred_df_conf{:.2f}.pickle'.format(CONF_THRESH)
if os.path.exists(OUTPUT_DIR+file_name):
    print ('Loading from file '+file_name)
    RFCNbbox_RFCNpred_df = pd.read_pickle(OUTPUT_DIR+file_name)
else:
    print ('Generating file '+file_name)        
    RFCNbbox_RFCNpred_df = pd.DataFrame(columns=['image_class','image_file','crop_index','xmin','ymin','xmax','ymax',
                                                  'NoF_RFCN', 'ALB_RFCN', 'BET_RFCN', 'DOL_RFCN',
                                                  'LAG_RFCN', 'OTHER_RFCN', 'SHARK_RFCN', 'YFT_RFCN']) 

    with open('../data/RFCN_detections/detections_full_AGNOSTICnms_'+RFCN_MODEL+'.pkl','rb') as f:
        detections_full_AGNOSTICnms = pickle.load(f) 
    with open("../RFCN/ImageSets/Main/test.txt","r") as f:
        test_files = f.readlines()
    with open("../RFCN/ImageSets/Main/train_test.txt","r") as f:
        train_file_labels = f.readlines()
    assert len(detections_full_AGNOSTICnms) == len(test_files)
    
    count = np.zeros(len(detections_full_AGNOSTICnms))
    
    for im in range(len(detections_full_AGNOSTICnms)):
        if im%1000 == 0: print(im)
        basename = test_files[im][:9]
        if im<1000:
            image_class = '--'
        else:
            for i in range(len(train_file_labels)):
                if train_file_labels[i][:9] == basename:
                    image_class = train_file_labels[i][10:-1]
                    break
        image = Image.open(TEST_DIR+'/'+basename+'.jpg')
        width_image, height_image = image.size
        
        bboxes = []
        detects_im = detections_full_AGNOSTICnms[im]
        for i in range(len(detects_im)):
            if np.sum(detects_im[i,5:]) >= CONF_THRESH:
                bboxes.append(detects_im[i,:]) 
        count[im] = len(bboxes)
        if len(bboxes) == 0:
            ind = np.argmax(np.sum(detects_im[:,5:], axis=1))
            bboxes.append(detects_im[ind,:])
        bboxes = np.asarray(bboxes)

        for j in range(len(bboxes)):    
            bbox = bboxes[j]
            xmin = bbox[0]
            ymin = bbox[1]
            xmax = bbox[2]
            ymax = bbox[3]
            assert max(xmin,0)<min(xmax,width_image)
            assert max(ymin,0)<min(ymax,height_image)
            RFCNbbox_RFCNpred_df.loc[len(RFCNbbox_RFCNpred_df)]=[image_class,basename+'.jpg',j,max(xmin,0),max(ymin,0),
                                                                   min(xmax,width_image),min(ymax,height_image),
                                                                   bbox[4],bbox[5],bbox[6],bbox[7],bbox[8],bbox[9],bbox[10],bbox[11]]   
    
    RFCNbbox_RFCNpred_df.to_pickle(OUTPUT_DIR+file_name)      

Loading from file RFCNbbox_RFCNpred_df_conf0.80.pickle


In [19]:
%%time
# RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df = ['image_class', 'image_file','crop_index','xmin','ymin','xmax','ymax',
#                                    'NoF_RFCN', 'ALB_RFCN', 'BET_RFCN', 'DOL_RFCN',
#                                    'LAG_RFCN', 'OTHER_RFCN', 'SHARK_RFCN', 'YFT_RFCN',
#                                    'NoF_CROP', 'ALB_CROP', 'BET_CROP', 'DOL_CROP',
#                                    'LAG_CROP', 'OTHER_CROP', 'SHARK_CROP', 'YFT_CROP',
#                                    'NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']

file_name = 'RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df_'+test_model_name+'_.pickle'
if os.path.exists(OUTPUT_DIR+file_name):
    print ('Loading from file '+file_name)
    RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df = pd.read_pickle(OUTPUT_DIR+file_name)
else:
    print ('Generating file '+file_name)  
    nb_augmentation = 1
    if nb_augmentation ==1:
        test_preds = test_model.predict_generator(test_generator(df=RFCNbbox_RFCNpred_df, mean=all_mean), 
                                                  steps=int(math.ceil(RFCNbbox_RFCNpred_df.shape[0]/BATCHSIZE)), workers=1, pickle_safe=False)
    else:
        test_preds = np.zeros((RFCNbbox_RFCNpred_df.shape[0], len(FISH_CLASSES)), dtype=K.floatx())
        for idx in range(nb_augmentation):
            print('{}th augmentation for testing ...'.format(idx+1))
            test_preds += test_model.predict_generator(test_generator(df=RFCNbbox_RFCNpred_df, mean=all_mean, datagen=test_aug_datagen), 
                                                       steps=int(math.ceil(RFCNbbox_RFCNpred_df.shape[0]/BATCHSIZE)), workers=1, pickle_safe=False)
        test_preds /= nb_augmentation

    CROPpred_df = pd.DataFrame(test_preds, columns=['NoF_CROP', 'ALB_CROP', 'BET_CROP', 'DOL_CROP', 'LAG_CROP', 'OTHER_CROP', 'SHARK_CROP', 'YFT_CROP'])
    RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df = pd.concat([RFCNbbox_RFCNpred_df,CROPpred_df], axis=1)
    
    RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df['NoF'] = RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df['NoF_RFCN']
    for fish in ['ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']:
        RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df[fish] = RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df.apply(lambda row: (1-row['NoF_RFCN'])*row[fish+'_CROP']/(1-row['NoF_CROP']), axis=1)

    RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df.to_pickle(OUTPUT_DIR+file_name) 

Generating file RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df_weights.139-0.0777.hdf5_.pickle
batch_index 0
CPU times: user 2min 45s, sys: 3.84 s, total: 2min 49s
Wall time: 2min


In [71]:
# clsMaxAve and hybrid RFCNpred&CROPpred such that RFCNpred for NoF and CROPpred for fish
# test_pred_df = ['logloss','image_class','image_file','NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT', 'split']
# RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df = ['image_class', 'image_file','crop_index','xmin','ymin','xmax','ymax',
#                                    'NoF_RFCN', 'ALB_RFCN', 'BET_RFCN', 'DOL_RFCN',
#                                    'LAG_RFCN', 'OTHER_RFCN', 'SHARK_RFCN', 'YFT_RFCN',
#                                    'ALB_CROP', 'BET_CROP', 'DOL_CROP',
#                                    'LAG_CROP', 'OTHER_CROP', 'SHARK_CROP', 'YFT_CROP',
#                                    'NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']

file_name = 'test_pred_df_Hybrid_'+test_model_name+'_.pickle'
if os.path.exists(OUTPUT_DIR+file_name):
    print ('Loading from file '+file_name)
    test_pred_df = pd.read_pickle(OUTPUT_DIR+file_name)
else:
    print ('Generating file '+file_name)  
    with open("../RFCN/ImageSets/Main/test.txt","r") as f:
        test_files = f.readlines()
    
    test_pred_df = pd.DataFrame(columns=['logloss','image_class','image_file','NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT'])  
    for j in range(len(test_files)): 
        image_file = test_files[j][:-1]+'.jpg'
        test_pred_im_df = RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df.loc[RFCNbbox_RFCNpred_CROPpred_HYBRIDpred_df['image_file'] == image_file,
                                                                       ['image_class', 'NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']]
        image_class = test_pred_im_df.iloc[0]['image_class']
        test_pred_im_df.drop('image_class', axis=1, inplace=True)
        max_score = test_pred_im_df.max(axis=1)
        max_cls = test_pred_im_df.idxmax(axis=1)
        test_pred_im_df['max_score'] = max_score
        test_pred_im_df['max_cls'] = max_cls
        test_pred_im_df['Count'] = test_pred_im_df.groupby(['max_cls'])['max_cls'].transform('count')
        idx = test_pred_im_df.groupby(['max_cls'])['max_score'].transform(max) == test_pred_im_df['max_score']
        test_pred_im_clsMax_df = test_pred_im_df.loc[idx,['NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT', 'Count']]
        test_pred_im_clsMax_array = test_pred_im_clsMax_df.values
        pred = np.average(test_pred_im_clsMax_array[:,:-1], axis=0, weights=test_pred_im_clsMax_array[:,-1], returned=False).tolist()
        if image_class!='--':
            ind = FISH_CLASSES.index(image_class)
            logloss = -math.log(pred[ind]) 
        else:
            logloss = np.nan
        test_pred_im_clsMaxAve = [logloss,image_class,image_file]
        test_pred_im_clsMaxAve.extend(pred)
        test_pred_df.loc[len(test_pred_df)]=test_pred_im_clsMaxAve
    
    image_split_df = GTbbox_df[['image_file','split']]
    image_split_df.drop_duplicates(inplace = True)
    test_pred_df = pd.merge(test_pred_df, image_split_df, how='outer', on='image_file')

    test_pred_df.to_pickle(OUTPUT_DIR+file_name) 

image_valid_loss = test_pred_df.loc[(test_pred_df['image_class']!='--') & (test_pred_df['split']=='valid'), 'logloss'].mean()
print('image_valid_loss:', image_valid_loss)
image_all_loss = test_pred_df.loc[test_pred_df['image_class']!='--', 'logloss'].mean()
print('image_all_loss:', image_all_loss)

Loading from file test_pred_df_Hybrid_weights.139-0.0777.hdf5_.pickle
image_valid_loss: 0.12416032312
image_all_loss: 0.0437935297746


In [76]:
all_IMAGEpred_df = test_pred_df[test_pred_df['image_class']!='--']
print('all image loss:', all_IMAGEpred_df['logloss'].mean())
print(all_IMAGEpred_df.groupby(['image_class'])['logloss'].mean())

train_IMAGEpred_df = all_IMAGEpred_df[all_IMAGEpred_df['split']=='train']
print('train image loss:', train_IMAGEpred_df['logloss'].mean())
print(train_IMAGEpred_df.groupby(['image_class'])['logloss'].mean())

valid_IMAGEpred_df = all_IMAGEpred_df[all_IMAGEpred_df['split']=='valid']
print('valid image loss:', valid_IMAGEpred_df['logloss'].mean())
print(valid_IMAGEpred_df.groupby(['image_class'])['logloss'].mean())

all image loss: 0.0437935297746
image_class
ALB      0.036905
BET      0.078541
DOL      0.008347
LAG      0.001009
NoF      0.126442
OTHER    0.011244
SHARK    0.073284
YFT      0.013843
Name: logloss, dtype: float64
train image loss: 0.0348822706301
image_class
ALB      0.023915
BET      0.016391
DOL      0.008006
LAG      0.000950
NoF      0.135392
OTHER    0.003376
SHARK    0.080938
YFT      0.010966
Name: logloss, dtype: float64
valid image loss: 0.12416032312
image_class
ALB      0.153740
BET      0.637887
DOL      0.011326
LAG      0.001518
NoF      0.044916
OTHER    0.081797
SHARK    0.001697
YFT      0.039891
Name: logloss, dtype: float64


In [79]:
#temperature
T = 2.5
test_pred_array = test_pred_df[FISH_CLASSES].values
test_pred_T_array = np.exp(np.log(test_pred_array)/T)
test_pred_T_array = test_pred_T_array/np.sum(test_pred_T_array, axis=1, keepdims=True)
test_pred_T_df = pd.DataFrame(test_pred_T_array, columns=FISH_CLASSES)
test_pred_T_df = pd.concat([test_pred_df[['image_class','image_file']],test_pred_T_df], axis=1)

#test submission
submission = test_pred_T_df.loc[:999,['image_file','NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']]
submission.rename(columns={'image_file':'image'}, inplace=True)
sub_file = 'RFCN_AGONOSTICnms_'+RFCN_MODEL+'_'+CROP_MODEL+'_'+test_model_name+ \
'_clsMaxAve_conf{:.2f}_cropvalidloss{:.4f}_cropallloss{:.4f}_imagevalidloss{:.4f}_imageallloss{:.4f}_T{}.csv' \
.format(CONF_THRESH, crop_valid_loss, crop_all_loss, image_valid_loss, image_all_loss, T)
submission.to_csv(OUTPUT_DIR + sub_file, index=False)
print('Done!'+sub_file)

Done!RFCN_AGONOSTICnms_resnet101_rfcn_ohem_iter_30000_resnet50_FT38_Hybrid_wNoF2_weights.139-0.0777.hdf5_clsMaxAve_conf0.80_cropvalidloss0.1038_cropallloss0.0196_imagevalidloss0.1242_imageallloss0.0438_T2.5.csv
